In [1]:
using JuMP, DataFrames

# Bulb Luminisity

fixed cost

In [17]:
# model
m = Model()

# variable

@variable(m, x[1:5]>=0)
@variable(m, y[1:5], Bin)

#costraint
for i = 1:5
    @constraint(m,x[i]<=300*y[i])
end

@constraint(m, x[1]+x[2]>=250)
@constraint(m, x[1]+x[3]>=300)
@constraint(m, x[2]+x[3]>=150)
@constraint(m, x[2]+x[4]>=200)
@constraint(m, x[3]+x[5]>=350)
@constraint(m, x[4]+x[5]>=180)

# objective
@objective(m,Min,sum{450*y[i]+2*x[i], i in 1:5})

450 y[1] + 2 x[1] + 450 y[2] + 2 x[2] + 450 y[3] + 2 x[3] + 450 y[4] + 2 x[4] + 450 y[5] + 2 x[5]

In [18]:
solve(m)

:Optimal

In [19]:
# Determine which items to carry 
println("Variable Values: ", getvalue(x))

# Determine value from items carried
println("Objetive value: ", getobjectivevalue(m))

Variable Values: [0.0,250.0,300.0,0.0,180.0]
Objetive value: 2810.0


In [20]:
# Determine which items to carry 
println("Variable Values: ", getvalue(y))

Variable Values: [0.0,1.0,1.0,0.0,1.0]


# Bulb Luminisity 2

fixed cost with more constrains

In [22]:
# model
m = Model()

# variable
@variable(m, x[1:5]>=0)
@variable(m, y[1:5], Bin)
@variable(m, z[1:5], Bin)
@variable(m, w, Bin)

#costraint
for i = 1:5
    @constraint(m,x[i]<=300*y[i])
    @constraint(m,x[i]<=300-150*z[i])
end

@constraint(m, x[1]+x[2]>=250)
@constraint(m, x[1]+x[3]>=300)
@constraint(m, x[2]+x[3]>=150)
@constraint(m, x[2]+x[4]>=200)
@constraint(m, x[3]+x[5]>=350)
@constraint(m, x[4]+x[5]>=180)
@constraint(m, x[1]>=300*w)
@constraint(m, x[4]>=300*(1-w))
@constraint(m, sum{z[i], i in 1:5}>=3)
@constraint(m, 0.6*x[1]-0.4*x[3]<=0)

# objective
@objective(m,Min,sum{450*y[i]+2*x[i], i in 1:5})

450 y[1] + 2 x[1] + 450 y[2] + 2 x[2] + 450 y[3] + 2 x[3] + 450 y[4] + 2 x[4] + 450 y[5] + 2 x[5]

In [23]:
solve(m)

:Optimal

In [24]:
# Determine which items to carry 
println("Variable Values: ", getvalue(y))

# Determine value from items carried
println("Objetive value: ", getobjectivevalue(m))

Variable Values: [0.9999999999999999,1.0,1.0,1.0,1.0]
Objetive value: 4050.0


# Production Plan

In [80]:
m = Model()

# Parameters
item=3
time_period=4
setup_time=[45, 60, 100]
price=[50, 70, 120]
production_rate=[5, 4, 2]
inventory_cost=[18, 15, 25]
demand=[400 600 200 800; 240 440 100 660; 80 120 40 100]
time_limit=480

480

In [81]:
#variable
@variable(m, z[1:item, 1:time_period], Bin) # 1 if item i is produced on day t, 0 otherwise
@variable(m, s[1:item, 1:time_period] >=0) # inventory of item i at the end of day t
@variable(m, y[1:item, 1:time_period] >=0) # number of units of item i produced on day t
@variable(m, x[1:item, 1:time_period] >=0) # time machine spends on day t prod

3x4 Array{JuMP.Variable,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]

In [82]:
#constrain
## machine use time
for t = 1:4
    @constraint(m,sum{setup_time[i]*z[i,t], i=1:3}+sum{x[i,t] ,i=1:3}<=480)
end

for i=1:3
    @constraint(m, s[i,4] == 0)
end

for t = 1:4
    for i = 1:3
        ## inventory balance
        if t==1
            @constraint(m,y[i,t]-demand[i,t]==s[i,t])
        else
            @constraint(m,s[i,t-1]+y[i,t]-demand[i,t]==s[i,t])
        end
        ## other
        @constraint(m,y[i,t]==production_rate[i]*x[i,t])
        @constraint(m,x[i,t]<=z[i,t]*480)
    end
end

In [83]:
# objective
@objective(m,Max,sum{demand[i,t]*price[i]-inventory_cost[i]*s[i,t], i=1:item, t=1:time_period})

-18 s[1,1] - 18 s[1,2] - 18 s[1,3] - 18 s[1,4] - 15 s[2,1] - 15 s[2,2] - 15 s[2,3] - 15 s[2,4] - 25 s[3,1] - 25 s[3,2] - 25 s[3,3] - 25 s[3,4] + 241600

In [84]:
solve(m)

:Optimal

In [86]:
println("Objective value: ", getObjectiveValue(m))
for i=1:item
    for t=1:time_period
        println("Produce Item ", i, ": ", getValue(y[i,t]), " at day ", t)
    end
end

Objective value: 238350.0
Produce Item 1: 400.0 at day 1
Produce Item 1: 600.0 at day 2
Produce Item 1: 200.0 at day 3
Produce Item 1: 800.0 at day 4
Produce Item 2: 240.00000000000003 at day 1
Produce Item 2: 440.00000000000006 at day 2
Produce Item 2: 100.00000000000001 at day 3
Produce Item 2: 660.0 at day 4
Produce Item 3: 110.0 at day 1
Produce Item 3: 90.0 at day 2
Produce Item 3: 140.00000000000003 at day 3
Produce Item 3: 0.0 at day 4


# Production Plan with Super Day

In [94]:
m = Model()

# Parameters
item=3
time_period=4
setup_time=[45, 60, 100]
price=[50, 70, 120]
production_rate=[5, 4, 2]
inventory_cost=[18, 15, 25]
demand=[400 600 200 800; 240 440 100 660; 80 120 40 100]
time_limit=480

#variable
@variable(m, z[1:item, 1:time_period], Bin) # 1 if item i is produced on day t, 0 otherwise
@variable(m, u[1:time_period], Bin) # 1 if it's super day
@variable(m, s[1:item, 1:time_period] >=0) # inventory of item i at the end of day t
@variable(m, y[1:item, 1:time_period] >=0) # number of units of item i produced on day t
@variable(m, x[1:item, 1:time_period] >=0) # time machine spends on day t prod

3x4 Array{JuMP.Variable,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]

In [95]:
#constrain
## machine use time
for t = 1:4
    @constraint(m,sum{setup_time[i]*z[i,t], i=1:3}+sum{x[i,t] ,i=1:3}<=480+120*u[t])
end

for i=1:3
    @constraint(m, s[i,4] == 0)
end

for t = 1:4
    for i = 1:3
        ## inventory balance
        if t==1
            @constraint(m,y[i,t]-demand[i,t]==s[i,t])
        else
            @constraint(m,s[i,t-1]+y[i,t]-demand[i,t]==s[i,t])
        end
        ## other
        @constraint(m,y[i,t]==production_rate[i]*x[i,t])
        @constraint(m,x[i,t]<=z[i,t]*480)
    end
end

# u
@constraint(m, sum{u[i],i=1:4}<=2)
for t = 2:4
    @constraint(m,u[t-1]+u[t]<=1)
end

In [96]:
# objective
@objective(m,Max,sum{demand[i,t]*price[i]-inventory_cost[i]*s[i,t], i=1:item, t=1:time_period}-50*sum{u[i],i=1:4})

-18 s[1,1] - 18 s[1,2] - 18 s[1,3] - 18 s[1,4] - 15 s[2,1] - 15 s[2,2] - 15 s[2,3] - 15 s[2,4] - 25 s[3,1] - 25 s[3,2] - 25 s[3,3] - 25 s[3,4] - 50 u[1] - 50 u[2] - 50 u[3] - 50 u[4] + 241600

In [97]:
solve(m)

:Optimal

In [98]:
println("Objective value: ", getObjectiveValue(m))
for i=1:item
    for t=1:time_period
        println("Produce Item ", i, ": ", getValue(y[i,t]), " at day ", t)
    end
end

Objective value: 241500.0
Produce Item 1: 400.00000000000006 at day 1
Produce Item 1: 599.9999999999999 at day 2
Produce Item 1: 200.0 at day 3
Produce Item 1: 800.0000000000001 at day 4
Produce Item 2: 239.99999999999997 at day 1
Produce Item 2: 440.0 at day 2
Produce Item 2: 100.0 at day 3
Produce Item 2: 659.9999999999999 at day 4
Produce Item 3: 80.0 at day 1
Produce Item 3: 120.00000000000001 at day 2
Produce Item 3: 40.00000000000001 at day 3
Produce Item 3: 99.99999999999999 at day 4


In [99]:
getValue(u)

4-element Array{Float64,1}:
 0.0
 1.0
 0.0
 1.0

# Sudokum

In [105]:
# model
m = Model()

# data
# The given digits
init_sol = [ 5 3 0 0 7 0 0 0 0;
6 0 0 1 9 5 0 0 0;
0 9 8 0 0 0 0 6 0;
8 0 0 0 6 0 0 0 3;
4 0 0 8 0 3 0 0 1;
7 0 0 0 2 0 0 0 6;
0 6 0 0 0 0 2 8 0;
0 0 0 4 1 9 0 0 5;
0 0 0 0 8 0 0 7 9]

# variable
@variable(m, x[1:9,1:9,1:9], Bin)

#costraint
for ind = 1:9 # Each row, OR each column
    for k = 1:9 # Each digit
        @constraint(m,sum{x[ind,j,k],j=1:9}==1)
        @constraint(m,sum{x[i,ind,k],i=1:9}==1)
    end
end

for i = 1:9, j = 1:9
    @constraint(m,sum{x[i,j,k],k=1:9}==1)
end

for i = 1:3:7, j = 1:3:7, k = 1:9
# i is the top left row, j is the top left column
# For each 3x3 square, we sum from from row i to i+2 and column j to j+2
    @constraint(m, sum{x[r,c,k], r=i:i+2, c=j:j+2} == 1)
end

for i = 1:9, j = 1:9
    # If the space isn’t empty
    if init_sol[i,j] != 0
    # Then the corresponding variable for that digit and location must be 1
    @constraint(m, x[i,j,init_sol[i,j]] == 1)
    end
end

In [106]:
solve(m)

:Optimal

In [107]:
# Get the values of x
x_val = getValue(x)
# Create a matrix to store the solution
sol = zeros(Int,9,9) # 9x9 matrix of integers
for i in 1:9, j in 1:9, k in 1:9
    if round(Int, x_val[i,j,k]) == 1
        sol[i,j] = k
    end
end
println(sol)

[5 3 4 6 7 8 9 1 2
 6 7 2 1 9 5 3 4 8
 1 9 8 3 4 2 5 6 7
 8 5 9 7 6 1 4 2 3
 4 2 6 8 5 3 7 9 1
 7 1 3 9 2 4 8 5 6
 9 6 1 5 3 7 2 8 4
 2 8 7 4 1 9 6 3 5
 3 4 5 2 8 6 1 7 9]


# Airline Network Revenue Management

In [108]:
# model
m = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

In [109]:
# variable
@variable(m, x[["b","m"],["m","s"],["r","d"]]>=0,Int)

x[i,j,k] ≥ 0 ∀ i ∈ {b,m}, j ∈ {m,s}, k ∈ {r,d}

In [111]:
# object
@objective(m, Max, 428*x["b","m","r"]+190*x["b","m","d"]+642*x["b","s","r"]+224*x["b","s","d"]+512*x["m","s","r"]+190*x["m","s","d"])

428 x[b,m,r] + 190 x[b,m,d] + 642 x[b,s,r] + 224 x[b,s,d] + 512 x[m,s,r] + 190 x[m,s,d]

In [112]:
# constrain
@constraint(m, x["b","m","r"]<=80)
@constraint(m, x["b","m","d"]<=120)
@constraint(m, x["b","s","r"]<=75)
@constraint(m, x["b","s","d"]<=100)
@constraint(m, x["m","s","r"]<=60)
@constraint(m, x["m","s","d"]<=110)

@constraint(m, x["b","m","r"]+x["b","m","d"]+x["b","s","r"]+x["b","s","d"]<=166)
@constraint(m, x["m","s","r"]+x["m","s","d"]+x["b","s","r"]+x["b","s","d"]<=166)

x[m,s,r] + x[m,s,d] + x[b,s,r] + x[b,s,d] ≤ 166

In [113]:
solve(m)

:Optimal

In [115]:
getObjectiveValue(m)

121090.0

In [116]:
getValue(x)

x: 3 dimensions:
[b,:,:]
  [b,m,:]
    [b,m,r] = 80.0
    [b,m,d] = 11.0
  [b,s,:]
    [b,s,r] = 75.0
    [b,s,d] = 0.0
[m,:,:]
  [m,m,:]
    [m,m,r] = 0.0
    [m,m,d] = 0.0
  [m,s,:]
    [m,s,r] = 60.0
    [m,s,d] = 31.0
